# Experiment and path specification

In [ ]:
# Specify the experiment for analysis
experiment='cxic00318'
runNumber = 123

# Set the installation and output path
import os
os.environ['INSTALLPATH']= '/cds/home/i/igabalsk/TRXS-Run18'
os.environ['OUTPUTPATH']= '/cds/data/psdm/%s/%s/scratch' % (experiment[0:3],experiment)

# Import Libraries

In [ ]:
# Magic iPython command to enable plotting
%matplotlib inline

# Load in the pythonBatchMagic library
import sys
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/pythonBatchMagic')
from pythonBatchMagic import *

# Determine current user
currentUser, error = unixCMD("echo $USER")
currentUser = currentUser.strip()
print(currentUser)
os.environ['RESULTSPATH']= ('/cds/data/psdm/%s/%s/results/%s' % (experiment[0:3],experiment,currentUser)).strip()
if not os.path.exists(os.environ['RESULTSPATH']):
    os.mkdir(os.environ['RESULTSPATH'])

# Leveraging the batch queue to quickly grab point data

## Loading libraries

In [ ]:
sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/LCLS')
from LCLSdefault import *

sys.path.insert(0, os.environ['INSTALLPATH']+'/Libraries/mattsLibraries')
from picklez import *

# Load in the get data library
from dataAnalysis import *

# Load in the batch library for lcls
from lclsBatch import *

## Grabbing point data across nodes

In [ ]:

# detDict = {'opaint':{'name':'XppEnds_Ipm0','get-function': 'getIPM' },
#            'xint3':{'name':'XppSb3_Ipm','get-function': 'getIPM' },
#            'xpos3':{'name':'XppSb3_Ipm','get-function': 'getXPos' },
#            'ypos3':{'name':'XppSb3_Ipm','get-function': 'getYPos' },
#            'xint2':{'name':'XppSb2_Ipm','get-function': 'getIPM' },
#            'xpos2':{'name':'XppSb2_Ipm','get-function': 'getXPos' },
#            'ypos2':{'name':'XppSb2_Ipm','get-function': 'getYPos' },
#            'xrayEnergy':{'name':'SIOC:SYS0:ML00:AO541','get-function': 'getXrayEnergy' },
#           'stageencoder':{'name':'XppEndstation.0:USDUSB.0','get-function':'getStageEncoder'},
#           'ttfltpos':{'name':'XPP:TIMETOOL:FLTPOS', 'get-function': 'getTTFltPos'},
#           'ebeamcharge':{'name':'EBeam', 'get-function': 'getEbeamCharge'},
#           'fltposfwhm':{'name':'XPP:TIMETOOL:FLTPOSFWHM', 'get-function': 'getFltPosFWHM'},
#           'seconds':{'name':'seconds', 'get-function': 'getSeconds'},
#           'nanoseconds':{'name':'nanonseconds', 'get-function': 'getNanoseconds'},
#           'fiducials':{'name':'fiducials', 'get-function': 'getFiducials'},
#           'cspadSum':{'name':'fiducials', 'get-function': 'getCSPADsum'},
#           'cspadMedian':{'name':'fiducials', 'get-function': 'getCSPADmedian'}}

detDict = {'xrayEnergy':{'name':'FEEGasDetEnergy','get-function': 'getXrayEnergy' },
#           'ebeamcharge':{'name':'EBeam', 'get-function': 'getEbeamCharge'},
          'seconds':{'name':'seconds', 'get-function': 'getSeconds'},
          'nanoseconds':{'name':'nanonseconds', 'get-function': 'getNanoseconds'},
          'fiducials':{'name':'fiducials', 'get-function': 'getFiducials'}}


batchThreads = batchPointDataGrabber( detDict, eventMax=1e4, experiment=experiment, runNumber=runNumber, rank=10 )
batchThreads.Queue = 'psanaq'
batchThreads.start()

## Check job status

In [ ]:
stdoutdata, stderrdata = bjobs()
print stdoutdata
print stderrdata

## Kill threads if they die

In [ ]:
# batchThreads.requestStop()
time.sleep(5)
stdoutdata, stderrdata = bkill(killAll=True)
print stdoutdata
print stderrdata

In [ ]:
batchThreads.status

In [ ]:
batchThreads.gather()

## Drop empty array values

In [ ]:
pointData = batchThreads.detArrays

def dropEmpty(pointData):

    filledBin = (pointData['seconds'] > 0)    

    for key in pointData.keys():
        pointData[key]=pointData[key][filledBin] 
        
    return pointData

def dropNaN(pointData):
    goodBin = np.ones(pointData['seconds'].shape, dtype=bool)
    for key in pointData.keys():
        goodBin = goodBin & ~np.isnan(pointData[key])
    for key in pointData.keys():
        pointData[key]=pointData[key][goodBin]
    return pointData

pointData_masked = dropNaN(dropEmpty(pointData))

## Save point data

In [ ]:
save_obj( pointData_masked, os.environ['RESULTSPATH']+'/pointData-run-%d' % runNumber )

## Plot histogram of result

In [ ]:
detArrays = pointData_masked

print detArrays['seconds'].size
fig2, ax2 = plt.subplots(1, 1)
ax2.hist(detArrays['xrayEnergy'], bins=100,normed=False, alpha=0.5);